In [150]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np 

In [151]:
def read(his_file,sim_num,fname): 

# Middle of the vegetation patch  
    jmid=50; imid=45; 
# detect the presence of vegetation patch to know its extents (Currently hardwired
    imin_plant=41 ; imax_plant=50 ; jmin_plant=45 ; jmax_plant=54   
    
    nc = netCDF4.Dataset(his_file)
    mask = nc.variables['mask_rho'][:]

# read x,y
    x = nc.variables['x_rho'][:,:]
    y = nc.variables['y_rho'][:,:] 
    tot_time = nc.variables['ocean_time'] 

# Remember all arrays are (time,kmax,jmax,imax) contrary to Matlab     
    imax=size(x[1,:])
    jmax=size(y[:,1])
    
# read water depth
    depth = nc.variables['h'][:,:]
    theta_s = nc.variables['theta_s'][:]
    theta_b = nc.variables['theta_b'][:]
    depth_c = nc.variables['hc'][:]
    
#Get Stretching function 
    N=10 # Number of vertical levels, kgrid=1 MATLAB code copy 
    Np=N+1 
    ds=1.0/N
    s=np.zeros(11)
    Csur=np.zeros(11)
    for i in range(0,Np):
        lev=i
        s[i]=(lev-N)*ds
        Csur[i]=-s[i]**2
    
# Reshape 1D vertical variables to 3D so we can broadcast
    s.shape=(np.size(s),1,1)
    Csur.shape=(np.size(Csur),1,1)
    
# read 3D water level at all time steps
    eta = nc.variables['zeta'] 
    
# calculate the 4D field of z values (vertical coordinate)
    z0=(depth_c*s+depth*Csur)/(depth_c+depth)
    
# Extract the time frame for the last tidal cycle
    t_tidal=[]
    for t in range(1,size(tot_time)):
        if(eta[t,jmid,imid]<0.0 and eta[t+1,jmid,imid]>0.0):
            t_tidal.append(t)

    t_tidal_srt=t_tidal[-2]
    t_tidal_end=t_tidal[-1]
    print "Last tidal cycle starts and ends at this instance" 
    print t_tidal_srt,t_tidal_end   
    t_slice=t_tidal_end-t_tidal_srt+1
    
# Find water level for all locations at last tidal cycle
    z=np.zeros(shape=(t_slice,Np,jmax,imax))
    for t in range(t_tidal_srt,t_tidal_end+1): 
        eta = nc.variables['zeta'][t,:,:] 
        z[t-t_tidal_srt,:,:,:] = eta+(eta+depth)*z0
    #print np.shape(z)

# Calculate wave dissipation due to vegetation just behind the patch 
# calculate it along a cross section at integrate along depth 
# Take 5 points on both sides of the edges 
# Note the values asymptote above 1 point on both sides of the edge 

    vol_dissip_veg=0.0 
    for t in range (t_tidal_srt,t_tidal_end+1): 
        dissip_veg =nc.variables['Dissip_veg'][t,:,:] 
        for j in range(jmin_plant-5,jmax_plant+6):
            for i in range(imin_plant-5,imax_plant+6):
                dy=y[j+1,i]-y[j,i]
                dx=x[j,i+1]-x[j,i]
                for k in range(1,Np): 
                    # change in water level at last time step 
                    dz = z[t-t_tidal_srt,k,j,i]-z[t-t_tidal_srt,k-1,j,i]          
                    vol_dissip_veg = dissip_veg[j,i]*dx*dy*dz+vol_dissip_veg#*dx*dy*dz + vol_dissip_veg 
    
    mean_vol_dissip_veg=vol_dissip_veg/t_slice

    print "mean_vol_dissip_veg"
    print mean_vol_dissip_veg 
    
    f.write("%s\t, %s\t \n" % (sim_num, mean_vol_dissip_veg))


In [152]:
'''Output from vegetation module COAWST 
Mean volume integrated wave dissipation around vegetation patch [(Watt/m2 * m3)/time = Watt-mts/second
'''

fname='veg_dissip_output.txt'
f = open(fname, 'w')
total_sim_list=16
f.write("Case_no, Dissipation(W-mts/sec): \n")

for sim_num in range(1,total_sim_list):
    path1='/media/gadar/DATADRIVE1/sensitivity_history/'
    path2='veg_test_his_'+ str(sim_num)+'.nc'
    url=path1+path2
    print url 
    read(url,sim_num,fname) 
    
f.close()

/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_1.nc
Last tidal cycle starts and ends at this instance
186 258
mean_vol_dissip_veg
24.8170119614
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_2.nc
Last tidal cycle starts and ends at this instance
186 258
mean_vol_dissip_veg
23.7770471604
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_3.nc
Last tidal cycle starts and ends at this instance
186 258
mean_vol_dissip_veg
24.6131673073
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_4.nc
Last tidal cycle starts and ends at this instance
186 258
mean_vol_dissip_veg
24.9723698096
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_5.nc
Last tidal cycle starts and ends at this instance
186 258
mean_vol_dissip_veg
24.6920894782
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_6.nc
Last tidal cycle starts and ends at this instance
186 258
mean_vol_dissip_veg
23.7015914415
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_7.nc
Last tidal cyc